In [ ]:
import sys
import vega
import os
from pathlib import Path

sys.path.append('/home/user/Review-Interpretable-VAEs/Review-Interpretable-VAEs/cloned_github_models/vega/vega-reproducibility/src')
import vanilla_vae
import train_vanilla_vae_suppFig1
import utils
from utils import *
from learning_utils import *
from vanilla_vae import VanillaVAE
from vega_model import VEGA

import sys
import os 
from pathlib import Path
sys.path.append('/home/BS94_SUR/phD/review/utils/utils_evaluation')
import utils_train_models
from utils_train_models import *

In [10]:
torch.backends.cudnn.enabled = True
dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', dev, flush=True)

Using device: cuda


In [3]:
data_path = '/home/BS94_SUR/phD/review/models reproductibility/pmvae/scripts/data/kang_count.h5ad'
pathway_file = '/home/BS94_SUR/phD/review/models reproductibility/VEGA/vega-reproducibility/data/reactomes.gmt'

In [4]:
save_model_path = '/home/BS94_SUR/phD/review/notebooks/review VAEs/perturbation/vega/model inference/saved model/'
path_to_save_embeddings = '/home/BS94_SUR/phD/review/notebooks/review VAEs/perturbation/vega/model inference/original/embeddings'
path_to_save_reconstructed = '/home/BS94_SUR/phD/review/notebooks/review VAEs/perturbation/vega/model inference/original/reconstructed'

In [5]:
name_model = 'VEGA'
name_dataset = 'kang'
column_labels_name = 'condition'

In [6]:
nb_training= 1
train_size = 0.9
select_hvg = False
n_top_genes = 2000
preprocess = False
exist_val_set = True
random_seed = 42

In [7]:
perturbation = 'original'

In [8]:
adata = sc.read(data_path)
pathway_dict = read_gmt(pathway_file, min_g=0, max_g=1000)
pathway_mask = create_pathway_mask(adata.var.index.tolist(), pathway_dict, add_missing=1, fully_connected=True)

In [11]:
torch.manual_seed(random_seed)
#Preprocessing 
prep_data = VAE_prepare_dataset(adata, column_labels_name, random_seed, name_model, train_size, pathway_file, preprocess, select_hvg, n_top_genes) 
adata = prep_data.preprocess_data(adata, name_model, preprocess, select_hvg, n_top_genes)
X, y = prep_data.extract_x_y_from_adata(adata, column_labels_name)
X_train,  X_test, labels_train,  labels_test = prep_data.split_data(X, y, train_size, random_seed)
y_train = prep_data.encode_y(labels_train)
y_test = prep_data.encode_y(labels_test)
index_train = prep_data.extract_index(X_train)
index_test = prep_data.extract_index(X_test)
adata_train, index_train = prep_data.build_adata_from_X(adata, index_train)
adata_test, index_test = prep_data.build_adata_from_X(adata, index_test)

if exist_val_set == True:
    X_train,  X_val, labels_train,  labels_val = prep_data.split_data(X_train, labels_train, train_size, random_seed)
    y_val = prep_data.encode_y(labels_val)
    index_val = prep_data.extract_index(X_val)
    adata_val, index_val = prep_data.build_adata_from_X(adata, index_val)

train_data, pathway_dict_train, pathway_mask_train = prep_data. build_vega_dataset(adata_train, y_train, pathway_file)
val_data, pathway_dict_val, pathway_mask_val = prep_data. build_vega_dataset(adata_val, y_val, pathway_file)
test_data, pathway_dict_test, pathway_mask_test = prep_data.build_vega_dataset(adata_test, y_test, pathway_file)

n = f'{perturbation}_seed_{random_seed}'

beta = 0.00005
dropout = 0.5
batch_size =64
lr = 1e-4
n_epochs = 800
train_p=25
test_p=25
pos_dec = True
save_path = save_model_path + f'trained_VEGA_{n}'

model_training = Vega_train_multiple_times(adata_train, adata_val, adata_test, name_model, name_dataset, n, train_data, val_data, test_data, n_epochs, lr, pathway_mask_train, batch_size, beta, dropout, train_p, test_p, pos_dec, dev, save_path, path_to_save_embeddings, path_to_save_reconstructed)
train_loader, val_loader, test_loader = model_training.build_data_loader(train_data, val_data, test_data, batch_size, name_model)
hist = model_training.train_VEGA(adata_train, adata_val, adata_test, train_loader, val_loader, n, n_epochs, lr, pathway_mask_train, dev, beta, save_path, dropout, pos_dec, train_p, test_p, path_to_save_embeddings, path_to_save_reconstructed, name_dataset)

0.5
[Epoch 1] | loss: 101.180 | test_loss: 73.564 |
[Epoch 2] | loss: 77.625 | test_loss: 62.244 |
[Epoch 3] | loss: 72.005 | test_loss: 59.464 |
[Epoch 4] | loss: 70.172 | test_loss: 58.128 |
[Epoch 5] | loss: 69.155 | test_loss: 57.103 |
[Epoch 6] | loss: 68.455 | test_loss: 56.003 |
[Epoch 7] | loss: 67.812 | test_loss: 55.280 |
[Epoch 8] | loss: 67.393 | test_loss: 54.779 |
[Epoch 9] | loss: 67.031 | test_loss: 54.342 |
[Epoch 10] | loss: 66.689 | test_loss: 53.918 |
[Epoch 11] | loss: 66.437 | test_loss: 53.740 |
[Epoch 12] | loss: 66.330 | test_loss: 53.208 |
[Epoch 13] | loss: 66.062 | test_loss: 52.781 |
[Epoch 14] | loss: 65.764 | test_loss: 52.904 |
[Epoch 15] | loss: 65.611 | test_loss: 52.564 |
[Epoch 16] | loss: 65.620 | test_loss: 52.126 |
[Epoch 17] | loss: 65.288 | test_loss: 51.982 |
[Epoch 18] | loss: 65.273 | test_loss: 51.720 |
[Epoch 19] | loss: 65.126 | test_loss: 51.531 |
[Epoch 20] | loss: 64.958 | test_loss: 51.636 |
[Epoch 21] | loss: 65.031 | test_loss: 51.23